### MACTORENDS DOWNLOADER

In this file, financial data from macrotrends.net are downloaded. This website contains a lot of financial data related to stocks. For the web app purposes, financial ratios are downloaded. 

In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import date
import time
import streamlit
import matplotlib.pyplot as plt
import re
import json
from bs4 import BeautifulSoup as bs

Firstly, we specify a variable "URL" which stores link that changes based on selected stock name. To present how the downloading works, we selected ticker for Apple, inc.

In [ ]:
def get_data_macro(stock_name):
    url = f'https://www.macrotrends.net/stocks/charts/{stock_name}/3m/financial-ratios?freq=Q'
    response = requests.get(url)
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    data = json.loads(p.findall(response.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    ratios = []
    for row in data:
        soup = bs(row['field_name'],'lxml')
        #soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        ratios.append(fields)
    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    df = pd.DataFrame(ratios, columns = headers)
    return df


In [7]:
stock_name="AAPL"
url = f'https://www.macrotrends.net/stocks/charts/{stock_name}/3m/financial-ratios?freq=Q'
url

'https://www.macrotrends.net/stocks/charts/AAPL/3m/financial-ratios?freq=Q'

In [10]:
response = requests.get(url)
response #200 = OK

<Response [200]>

In [11]:
p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
data = json.loads(p.findall(response.text)[0])

In [ ]:
data

In [13]:
headers = list(data[0].keys())
headers.remove('popup_icon')

In [14]:
headers

['field_name',
 '2021-09-30',
 '2020-09-30',
 '2019-09-30',
 '2018-09-30',
 '2017-09-30',
 '2016-09-30',
 '2015-09-30',
 '2014-09-30',
 '2013-09-30',
 '2012-09-30',
 '2011-09-30',
 '2010-09-30',
 '2009-09-30',
 '2008-09-30',
 '2007-09-30',
 '2006-09-30',
 '2005-09-30']

In [15]:
ratios = []

In [16]:
for row in data:
    soup = bs(row['field_name'])
    field_name = soup.select_one('a, span').text
    fields = list(row.values())[2:]
    fields.insert(0, field_name)
    ratios.append(fields)

pd.option_context('display.max_rows', None, 'display.max_columns', None)
df = pd.DataFrame(ratios, columns = headers)
print(df.head())

                 field_name 2021-09-30 2020-09-30 2019-09-30 2018-09-30  \
0             Current Ratio    1.07460    1.36360    1.54010    1.13290   
1  Long-term Debt / Capital    0.63360    0.60160    0.50360    0.46660   
2         Debt/Equity Ratio    1.97680    1.72080    1.19400    1.06850   
3              Gross Margin   41.77940   38.23330   37.81780   38.34370   
4          Operating Margin   29.78240   24.14730   24.57200   26.69400   

  2017-09-30 2016-09-30 2015-09-30 2014-09-30 2013-09-30 2012-09-30  \
0    1.27610    1.35270    1.10880    1.08010    1.67860    1.49580   
1    0.42030    0.37030    0.30880    0.20630    0.12070              
2    0.86300    0.67860    0.53900    0.31640    0.13730              
3   38.46990   39.07600   40.05990   38.58800   37.62450   43.87120   
4   26.76040   27.83540   30.47730   28.72230   28.66950   35.29600   

  2011-09-30 2010-09-30 2009-09-30 2008-09-30 2007-09-30 2006-09-30 2005-09-30  
0    1.60840    2.01130    2.74250    2.6

Here is the final dataset that contain financial ratio (on quarterly basis) for selected ticker AAPL.

In [18]:
df

,field_name,2021-09-30,2020-09-30,2019-09-30,2018-09-30,2017-09-30,2016-09-30,2015-09-30,2014-09-30,2013-09-30,2012-09-30,2011-09-30,2010-09-30,2009-09-30,2008-09-30,2007-09-30,2006-09-30,2005-09-30
0,Current Ratio,1.07460,1.36360,1.54010,1.13290,1.27610,1.35270,1.10880,1.08010,1.67860,1.49580,1.60840,2.01130,2.74250,2.64110,2.36590,2.25190,2.95380
1,Long-term Debt / Capital,0.63360,0.60160,0.50360,0.46660,0.42030,0.37030,0.30880,0.20630,0.12070,,,,,,,,
2,Debt/Equity Ratio,1.97680,1.72080,1.19400,1.06850,0.86300,0.67860,0.53900,0.31640,0.13730,,,,,,,,
3,Gross Margin,41.77940,38.23330,37.81780,38.34370,38.46990,39.07600,40.05990,38.58800,37.62450,43.87120,40.47890,39.37750,40.13990,35.20050,33.16790,28.98270,29.01440
4,Operating Margin,29.78240,24.14730,24.57200,26.69400,26.76040,27.83540,30.47730,28.72230,28.66950,35.29600,31.21510,28.18710,27.36280,22.21070,17.93070,12.70000,11.79380
5,EBIT Margin,29.78240,24.14730,24.57200,26.69400,26.76040,27.83540,30.47730,28.72230,28.66950,35.29600,31.21510,28.18710,27.36280,22.21070,17.93070,12.70000,11.79380
6,EBITDA Margin,32.86700,28.17480,29.39460,30.79920,31.19130,32.70700,35.29380,33.06930,32.62300,37.38980,32.89080,29.76160,29.07350,23.53370,19.26110,13.86490,13.07870
7,Pre-Tax Profit Margin,29.85290,24.43980,25.26660,27.44890,27.95790,28.46050,31.02710,29.25850,29.34590,35.62950,31.59850,28.42470,28.12260,23.86440,20.36780,14.58970,12.97830
8,Net Profit Margin,25.88180,20.91360,21.23810,22.41420,21.09240,21.18680,22.84580,21.61440,21.67050,26.66510,23.94660,21.48410,19.19360,16.32130,14.22000,10.29770,9.53270
9,Asset Turnover,1.04220,0.84760,0.76860,0.72620,0.61080,0.67030,0.80500,0.78850,0.82570,0.88890,0.93020,0.86750,0.90320,1.03650,0.96970,1.12260,1.20970
